In [33]:
!pip -q install datasets tiktoken openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [1]:
!pip install python-multipart

In [2]:
!pip install kaleido

In [3]:
!pip install fastapi

In [4]:
!pip install frontend

In [5]:
!pip install fitz

In [8]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 43.5 MB/s eta 0:00:00


In [27]:
with open('Code-pénal-marocain.txt', 'r', encoding='utf-8') as file:
    org_text = file.read()

def extract_articles(original_text):
    final_text = ""

    for paragraph in original_text.split('\n\n'):
        if 'Article ' in paragraph:
            article_start = paragraph.index('Article ')
            final_text += paragraph[article_start:] + '\n\n'

    return final_text.strip()


cleaned_text = extract_articles(org_text)

articles_list = cleaned_text.split('\n\n')


articles_clone = []

for i in range(len(articles_list)):
    if len(articles_list[i]) > 24:
        articles_clone.append(articles_list[i])

with open("penal_clean.txt", "w", encoding="utf-8") as f:
    for article in articles_clone:
        f.write(f"{article}\n\n")

In [39]:
import json

# Initialize an empty list to hold structured data
structured_data = []

with open('penal_clean.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    articles = content.split("Article ")[1:]  # Split and ignore the first empty result from the split

    for article in articles:
        number, *body = article.split('\n', 1)  # Split once at the first newline to separate article number from body
        structured_data.append({
            "id": f"article_{number}",
            "prompt": f"Article {number}",
            "completion": body[0].strip() if body else ""
        })

# Save the structured data to a JSON file
with open('structured_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(structured_data, json_file, ensure_ascii=False, indent=2)
with open('structured_data.json', 'r') as jsn_file:
  original_json = json.load(jsn_file)


In [35]:
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [34]:
!pip install tiktoken

In [42]:
def transform_json_list(original_json_list):
    # Initialize the transformed JSON with the system message
    transformed_json = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."}
        ]
    }

    # Iterate over each element in the original JSON list
    for item in original_json_list:
        # Append a user message with the prompt
        transformed_json["messages"].append({"role": "user", "content": item["prompt"]})
        # Append an assistant message with the completion
        transformed_json["messages"].append({"role": "assistant", "content": item["completion"]})

    return transformed_json

training_data = transform_json_list(original_json)

{'messages': [{'role': 'system', 'content': 'You are an assistant that occasionally misspells words'}, {'role': 'user', 'content': 'Article 1'}, {'role': 'assistant', 'content': "Est approuvé le texte formant code pénal tel qu'il est publié en\nannexe au présent dahir."}, {'role': 'user', 'content': 'Article 2'}, {'role': 'assistant', 'content': "Les dispositions de ce code recevront leur application dans toute\nl'étendue du Royaume à dater du 17 juin 1963.\n    Toutefois, celles de ces dispositions relatives au placement judiciaire\ndans un établissement thérapeutique et au placement judiciaire dans une\ncolonie agricole, mesures de sûreté personnelles prévues par les articles\n80 à 85 dudit code, ne recevront application que lorsque leur mise en\nvigueur aura été spécialement décidée par des arrêtés conjoints des\ndivers ministres intéressés."}, {'role': 'user', 'content': 'Article 4'}, {'role': 'assistant', 'content': "Les dispositions de ce code s'appliquent même aux matières réglé